In [ ]:
import numpy as np
import struct 
import os
import plotly.graph_objects as go
import time
from copy import deepcopy
from scipy.stats import chi2
import matplotlib.pyplot as plt

import lgchimera.general as general
from lgchimera.io import read_lidar_bin, read_gt
from lgchimera.pose_graph import PoseGraph

%load_ext autoreload
%autoreload 2

In [ ]:
np.set_printoptions(suppress=True, precision=3)

In [ ]:
kitti_seq = '0027'
start_idx = 0

Ground truth trajectory

In [ ]:
gtpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
gt_data = read_gt(gtpath)
gt_data = gt_data[start_idx:]
lla = gt_data[:,:3]

In [ ]:
from lgchimera.general import lla_to_ecef, ecef2enu

ref_lla = lla[0]
ecef = lla_to_ecef(*lla[0])
gt_enu = np.zeros((len(lla),3))

for i in range(len(lla)):
    ecef = lla_to_ecef(*lla[i])
    gt_enu[i] = ecef2enu(ecef[0], ecef[1], ecef[2], ref_lla[0], ref_lla[1], ref_lla[2])

gt_enu = gt_enu[:,[1,0,2]]

In [ ]:
from scipy.spatial.transform import Rotation as R

heading = gt_data[0][5] # heading angle
r = R.from_euler('XYZ', [0, 0, heading])
R_heading = r.as_matrix()

In [ ]:
N = len(gt_enu)
gt_traj = go.Scatter(x=gt_enu[:,0], y=gt_enu[:,1], hovertext=np.arange(N), name='Ground-truth')
fig = go.Figure(data=[gt_traj])
fig.update_layout(width=900, height=700, scene=dict(aspectmode='data'))
fig.show()

Load LiDAR ICP results

In [ ]:
# Load registration results from file
data_path = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'results', 'p2pl_icp')
ds_rate = 10
Q_ini = 0.01
run_name = 'start_{}_ds_{}_Q_ini_{}'.format(start_idx, ds_rate, Q_ini)
lidar_Rs = np.load(os.path.join(data_path, 'lidar_Rs_'+run_name+'.npy'))
lidar_ts = np.load(os.path.join(data_path, 'lidar_ts_'+run_name+'.npy'))
positions = np.load(os.path.join(data_path, 'positions_'+run_name+'.npy'))
lidar_covariances = np.load(os.path.join(data_path, 'covariances_'+run_name+'.npy'))
N = len(lidar_Rs)

In [ ]:
lidar_traj = go.Scatter(x=positions[:,0], y=positions[:,1], hovertext=np.arange(N), name='Lidar odometry')
gt_traj = go.Scatter(x=gt_enu[:N,0], y=gt_enu[:N,1], hovertext=np.arange(N), name='Ground-truth')
fig = go.Figure(data=[gt_traj, lidar_traj])
fig.update_layout(width=900, height=700, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
# gt_traj3d = go.Scatter3d(x=gt_enu[:,0], y=gt_enu[:,1], z=gt_enu[:,2], marker=dict(size=2), hovertext=np.arange(N), name='Ground-truth')
# lidar_traj3d = go.Scatter3d(x=positions[:,0], y=positions[:,1], z=positions[:,2], marker=dict(size=2), hovertext=np.arange(N), name='Lidar odometry')
# fig = go.Figure(data=[gt_traj3d, lidar_traj3d])
# fig.update_layout(width=900, height=700, scene=dict(aspectmode='data'))
# fig.show()

## Sliding window-based optimization

In [ ]:
GPS_SIGMA = 1.5 # [m] (In practice closer to 5)
GPS_INFO = np.eye(6)
GPS_INFO[:3,:3] *= 1/GPS_SIGMA**2

# Information scaling factor
#INFO_SCALE = 0.1
#GPS_INFO *= INFO_SCALE   
GPS_INFO *= 0.001
GPS_INFO_AUTH = 100 * GPS_INFO

# Lidar information: fixed vs adaptive
LIDAR_INFO_FIXED = True
LIDAR_INFO = 100 * np.eye(6)
LIDAR_INFO[:3,:3] *= 10 

#LIDAR_INFO *= INFO_SCALE

GPS_RATE = 10  # Ratio of lidar to GPS measurements
MAX_BIAS = 50  # Spoofing maximum bias [m]

In [ ]:
np.random.seed(0)
N_TRAJ = N  # Total length of sequence 
N_WINDOW = 100  # Window size
N_EPOCH = 1800  # Length of Chimera epoch

In [ ]:
gps_noises = np.random.normal(0, GPS_SIGMA, (N_TRAJ, 3)) 

In [ ]:
# Compute threshold
alpha = 0.001  # False alarm (FA) rate
T = chi2.ppf(1-alpha, df=3*N_WINDOW/GPS_RATE)
print("Threshold = ", T)

TEST_STAT_SCALE = 100.0

In [ ]:
# GPS rate
GPS_RATE = 10  # Ratio of lidar to GPS measurements

g = PoseGraph()

# Add initial node
R_abs = R_heading.copy()
t_abs = gt_enu[0].copy()
g.add_node(1, (R_abs, t_abs))

# Form spoofing attack
gps_spoofing_biases = np.zeros(N_TRAJ)  
attack_start_idx = N_TRAJ // 2
#gps_spoofing_biases[attack_start_idx:] = np.linspace(0, MAX_BIAS, N_nodes-attack_start_idx)  # Ramping attack
gps_spoofing_biases[attack_start_idx:] = MAX_BIAS  # Jump attack

# Simulate initial GPS measurement
gps_pos = gt_enu[0] + gps_noises[0]
gps_pos[0] += gps_spoofing_biases[0]
# Add GPS factor and edge
g.add_factor(1, (R_abs, gps_pos), information=GPS_INFO)

# Authentication variables
last_auth_idx = 0  # Index of last Chimera authentication
auth = True  # Authentication status

# Store results
graph_positions = []
gps_measurements = []
fgo_losses = []
q_gpss = []
q_lidars = []

# For each new frame
for k in range(1, N):
    start_time = time.time()

    # Get LiDAR odometry
    R_hat = np.array(lidar_Rs[k-1])
    t_hat = np.array(lidar_ts[k-1])

    # Initialize new node with LiDAR odometry estimate
    R_abs = R_hat @ R_abs
    t_abs += R_abs @ t_hat
    g.add_node(k+1, (R_abs, t_abs))

    # Add LiDAR odometry edge
    if LIDAR_INFO_FIXED:
        lidar_information = LIDAR_INFO
    else:
        lidar_information = np.linalg.inv(lidar_covariances[k-1])
    g.add_edge([k, k+1], (R_hat, t_hat), information=lidar_information)

    if k % GPS_RATE == 0:

        # Check for authentication
        if k % N_EPOCH == 0:
            print("------- CHIMERA AUTHENTICATION -------")
            #auth = True
            # Set authentication information
            #g.set_factor_informations(start=last_auth_idx, end=k, information=GPS_INFO_AUTH)
            last_auth_idx = k

        # Simulate GPS measurement
        gps_pos = gt_enu[k] + gps_noises[k]
        gps_pos[0] += gps_spoofing_biases[k]
        gps_measurements.append(gps_pos)

        # Add GPS factor and edge
        if auth:
            g.add_factor(k+1, (R_abs, gps_pos), information=GPS_INFO)
        else:
            g.add_factor(k+1, (R_abs, gps_pos), information=np.zeros((6,6)))

        # Trim to window size
        graph_size = max([v.id for v in g.graph._vertices])
        if graph_size > N_WINDOW:
            # Save pose before trimming
            graph_positions.append(g.get_positions()[:GPS_RATE])
            g.trim_window(n=GPS_RATE)

        # Compute test statistic
        q_gps, q_lidar = g.test_statistic()
        q_gpss.append(q_gps)
        q_lidars.append(q_lidar)
        #print("     q=", q)

        # Check if attack is detected
        if TEST_STAT_SCALE * q_gps > T:
            print("ATTACK DETECTED: k =", k)
            auth = False
            # Spoofing mitigation
            g.set_factor_informations(start=last_auth_idx, end=k, information=np.zeros((6,6)))

        # Optimize
        g.optimize(verbose=False)

        # Update latest pose
        R_abs, t_abs = g.get_poses()[-1]

        # Time each iteration
        print(k, "/", N_TRAJ, ": t=", time.time() - start_time, "s")

graph_positions = np.array(graph_positions).reshape(-1,3)
gps_measurements = np.array(gps_measurements)

In [ ]:
# fig = go.Figure(data=g.plot_trace())
# fig.update_layout(width=1600, height=900, scene=dict(aspectmode='data'))
# fig.show()

In [ ]:
q_gpss = q_gpss[:350]

In [ ]:
# Plot test statistic in plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(q_gpss)), y=100*np.array(q_gpss), name='GPS test statistic'))
#fig.add_trace(go.Scatter(x=np.arange(len(q_gpss)), y=np.array(q_lidars), name='LiDAR test statistic'))
fig.add_trace(go.Scatter(x=np.arange(len(q_gpss)), y=T*np.ones(len(q_gpss)), name='Threshold', line=dict(color='red', dash='dash')))
# Add vertical line at start of spoofing attack
fig.add_shape(type="line", x0=attack_start_idx/GPS_RATE, y0=-20, x1=attack_start_idx/GPS_RATE, y1=250, line=dict(color="black", width=2, dash="dash"))
fig.update_layout(width=1000, height=500, xaxis_title="Time (s)", yaxis_title="Test statistic")
fig.update_layout(legend=dict(x=0.78, y=0.98), font=dict(size=15))
fig.show()

In [ ]:
N_TRAJ = len(gt_enu)

In [ ]:
gps_spoofing_biases = np.zeros(N_TRAJ)  
attack_start_idx = N_TRAJ // 2
#gps_spoofing_biases[attack_start_idx:] = np.linspace(0, 1, N_TRAJ-attack_start_idx)  # Ramping attack
gps_spoofing_biases[attack_start_idx:] = 1

# 0027 traj len: 7:35 min = 455 sec
# Spoofing duration = 455 / 2 = 227.5 sec
# 0.1 m/s => 22.75 m total bias
# 0.2 m/s => 45.5 m total bias
# 0.5 m/s => 113.75 m total bias
# 10 m total bias = 10 / 227.5 = 0.044 m/s
# 20 m total bias = 20 / 227.5 = 0.088 m/s
# 50 m total bias = 50 / 227.5 = 0.22 m/s
spoof_pos_1 = gt_enu.copy()
spoof_pos_1[:,0] += 22.75 * gps_spoofing_biases
spoof_pos_2 = gt_enu.copy()
spoof_pos_2[:,0] += 45.5 * gps_spoofing_biases
spoof_pos_3 = gt_enu.copy()
spoof_pos_3[:,0] += 113.75 * gps_spoofing_biases

In [ ]:
fgo_traj = go.Scatter(x=graph_positions[:,0], y=graph_positions[:,1], hovertext=np.arange(N), name='FGO trajectory')
gt_traj = go.Scatter(x=gt_enu[:N-N_WINDOW,0], y=gt_enu[:N-N_WINDOW,1], hovertext=np.arange(N), name='Ground-truth')
# spoof_traj_1 = go.Scatter(x=spoof_pos_1[:N-N_WINDOW,0], y=spoof_pos_1[:N-N_WINDOW,1], hovertext=np.arange(N), name='0.1 m/s Spoofed GPS trajectory', line=dict(color='red', dash='dot'))
# spoof_traj_2 = go.Scatter(x=spoof_pos_2[:N-N_WINDOW,0], y=spoof_pos_2[:N-N_WINDOW,1], hovertext=np.arange(N), name='0.2 m/s Spoofed GPS trajectory', line=dict(color='red', dash='dash'))
# spoof_traj_3 = go.Scatter(x=spoof_pos_3[:N-N_WINDOW,0], y=spoof_pos_3[:N-N_WINDOW,1], hovertext=np.arange(N), name='0.5 m/s Spoofed GPS trajectory', line=dict(color='red', dash='dashdot'))

start = go.Scatter(x=[0], y=[0], name='Start', mode='markers', marker=dict(size=10, color='blue'), showlegend=False)
fig = go.Figure(data=[gt_traj, fgo_traj, start])
fig.update_layout(width=1000, height=1000, xaxis_title='East [m]', yaxis_title='North [m]')
# Move legend into plot
fig.update_layout(legend=dict(x=0.57, y=0.98), font=dict(size=18))
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
fig.update_xaxes(autorange=True)
fig.show()

In [ ]:
lidar_traj = go.Scatter(x=graph_positions[:,0], y=graph_positions[:,1], hovertext=np.arange(N), name='Factor graph')
gt_traj = go.Scatter(x=gt_enu[:N_nodes,0], y=gt_enu[:N_nodes,1], hovertext=np.arange(N), name='Ground-truth')
fig = go.Figure(data=[gt_traj, lidar_traj])
fig.update_layout(width=900, height=700, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
# RMSE error
rmse_xyz = np.sqrt(np.mean((graph_positions - gt_enu[:N_nodes-window_size])**2, axis=0))
print("RMSE (xyz): ", rmse_xyz)
print("RMSE (overall): ", rmse_xyz.mean())

In [ ]:
import plotly.express as px
# lidar_traj = go.Scatter(x=graph_positions[:,0], y=graph_positions[:,1], hovertext=np.arange(N), name='Factor graph')
# gt_traj = go.Scatter(x=gt_ecef[:N_nodes,0], y=gt_ecef[:N_nodes,1], hovertext=np.arange(N), name='Ground-truth', marker=dict(color=qs))
# fig = go.Figure(data=[gt_traj, lidar_traj])
fig = px.scatter(gt_ecef[1:N_nodes], x=0, y=1, color=qs)
fig.update_layout(width=900, height=700, scene=dict(aspectmode='data'))
fig.show()

## Segmented window-based optimization

In [ ]:
GPS_SIGMA = 1.5 # [m] (In practice closer to 5)
GPS_INFO = np.eye(6)
GPS_INFO[:3,:3] *= 1/GPS_SIGMA**2
GPS_INFO[3:,3:] *= 1  # Scale rotation component 

In [ ]:
# Lidar information: fixed vs adaptive
LIDAR_INFO_FIXED = False
LIDAR_INFO = np.eye(6)
LIDAR_INFO[:3,:3] *= 10 

# Downsample rate

# Lidar GPS relative weighting
LIDAR_GPS_WEIGHT = 1.0

In [ ]:
N_nodes = 1000
window_size = 100
num_windows = N_nodes // window_size

# Compute threshold
alpha = 0.001
T = chi2.ppf(1-alpha, df=3*window_size)
print("Threshold = ", T)

# Initial pose
R_abs = R_heading.copy()
t_abs = gt_ecef[0].copy()

gps_spoofing_biases = np.zeros(N_nodes)  
max_bias = 100
#gps_spoofing_biases[500:] = np.linspace(0, max_bias, 500)  # Ramping attack

graph_positions = []
gps_measurements = []
qs = []
lidar_trajs = []

# For each window
for i in range(num_windows):
    # Form graph over window
    g = PoseGraph()

    # Add initial node
    g.add_node(1, (R_abs, t_abs))

    # Simulate GPS measurement
    gps_pos = gt_ecef[i*window_size] + np.random.normal(0, GPS_SIGMA, 3) 
    gps_pos[0] += gps_spoofing_biases[i*window_size]
    # Add GPS factor and edge
    g.add_factor(1, (np.eye(3), gps_pos), information=GPS_INFO)

    for j in range(1,window_size):
        idx = i*window_size + j
        # Get LiDAR odometry
        R_hat = np.array(lidar_Rs[idx-1])
        t_hat = np.array(lidar_ts[idx-1])

        # Initialize new node with LiDAR odometry estimate
        R_abs = R_hat @ R_abs
        t_abs += R_abs @ t_hat
        g.add_node(j+1, (R_abs, t_abs))

        # Add LiDAR odometry edge
        if LIDAR_INFO_FIXED:
            lidar_information = LIDAR_INFO
        else:
            lidar_information = np.linalg.inv(lidar_covariances[idx-1])
        g.add_edge([j, j+1], (R_hat, t_hat), information=LIDAR_GPS_WEIGHT*lidar_information)

        # Simulate GPS measurement
        gps_pos = gt_ecef[idx] + np.random.normal(0, GPS_SIGMA, 3) 
        gps_pos[0] += gps_spoofing_biases[idx]
        gps_measurements.append(gps_pos)

        # Add GPS factor and edge 
        g.add_factor(j+1, (R_abs, gps_pos), information=GPS_INFO)

    lidar_trajs.append(g.get_positions())

    # Compute test statistic
    q = g.test_statistic()
    qs.append(q)

    # Optimize
    g.optimize()

    # Store positions
    graph_positions.append(g.get_positions())

    # Update R_abs and t_abs
    R_abs, t_abs = g.get_poses()[-1]

    # Time each iteration
    print("window", i, "/", num_windows)

graph_positions = np.reshape(graph_positions, (-1,3))
gps_measurements = np.asarray(gps_measurements)

In [ ]:
qs

In [ ]:
plt.plot(qs)
#plt.plot(T*np.ones(len(qs)), 'r--')
plt.title("Test statistic vs time")
plt.xlabel("Time (frames)")
plt.ylabel("Test statistic")
plt.show()

In [ ]:
for lidar_traj in lidar_trajs:
    plt.plot(lidar_traj[:,0], lidar_traj[:,1], 'b')

plt.plot(gps_measurements[:,0], gps_measurements[:,1], 'r')

In [ ]:
lidar_traj = go.Scatter(x=graph_positions[:,0], y=graph_positions[:,1], hovertext=np.arange(N), name='Factor graph')
gt_traj = go.Scatter(x=gt_ecef[:N_nodes,0], y=gt_ecef[:N_nodes,1], hovertext=np.arange(N), name='Ground-truth')
gps = go.Scatter(x=gps_measurements[:,0], y=gps_measurements[:,1], hovertext=np.arange(N), name='GPS measurements')
fig = go.Figure(data=[gt_traj, lidar_traj, gps])
fig.update_layout(width=900, height=700, scene=dict(aspectmode='data'))
fig.show()